# import Peckage

In [ ]:
!pip install optuna
!pip install transformers datasets evaluate
!pip install transformers datasets[audio]
!pip install jiwer
!pip install accelerate -U
!pip install librosa
!pip install soundfile

In [ ]:
import pandas as pd
import IPython.display as ipd

from scipy import signal
from scipy.io import wavfile

from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq


import librosa
import librosa.display

import soundfile as sf
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

from datasets import load_dataset, Audio
from transformers import AutoProcessor

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load Data

In [ ]:
from datasets import load_dataset, DatasetDict

minds = load_dataset("PolyAI/minds14",'en-US', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Exploratory data analysis (EDA)

In [ ]:
def eda_plots(sample_data):

    data, samplerate = librosa.load(sample_data['path'], sr = sample_data['audio']['sampling_rate'])
    sample_audio = sample_data["audio"]

    # 1. plot the waveform

    fig, axs = plt.subplots(4,1,figsize=(18,15), gridspec_kw={'height_ratios': [1, 1, 1, 1.75]})
    axs[0].set_title('Waveform')
    axs[0].set_ylabel('Amplitude')
    axs[0].set_xlabel('Time (s)')
    librosa.display.waveshow(data, sr=samplerate, ax=axs[0])
    axs[0].set_xlim((0, len(data)/samplerate))


    # 2. plot the Spectogram

    # # IF using mel scale
    # spectogram = librosa.feature.melspectrogram(y=data, sr=samplerate)        # Compute the spectrogram using Mel scale
    # spectogram_log = librosa.power_to_db(spectogram,  ref=np.max)                 # Convert power spectrogram to dB scale
    # librosa.display.specshow(xdb, sr=samplerate, x_axis='time', y_axis='mel', ax=axs[1], cmap= 'magma')


    # # IF using Hertz
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))

    librosa.display.specshow(xdb, sr=samplerate, x_axis='time', y_axis='hz', ax=axs[1], cmap= 'viridis')
    axs[1].set_title('Spectrogram')
    axs[1].set_ylabel('Frequency (Hz)')
    axs[1].set_xlabel('Time (s)')
    # plt.colorbar(spect, cax= axs[1], label='Intensity (dB)')
    axs[1].set_xlim((0, len(data)/samplerate))


    # 3. plot MFCC

    mfccs = librosa.feature.mfcc(y=data, sr=samplerate, n_mfcc=13)
    librosa.display.specshow(mfccs, x_axis='time', sr=samplerate, ax=axs[2], cmap= 'viridis')

    axs[2].set_xlabel('Time (s)')
    axs[2].set_ylabel('MFCC Coefficients')
    axs[2].set_title('MFCCs')


    # 4. plot the FFT

    N = len(data)
    x = np.linspace(0, N/samplerate, N, endpoint=False)
    xf = fftfreq(N, 1/samplerate)[:N//2]
    yf = fft(data)
    axs[3].plot(xf, 2.0/N * np.abs(yf[0:N//2]))
    axs[3].set_title('Fast Fourier Transform (FFT)')
    axs[3].set_ylabel('Magnitude')
    axs[3].set_xlabel('Frequency (Hz)')
    axs[3].set_xlim((0, max(xf)+5))

    plt.tight_layout()
    plt.show()


    plt.tight_layout()
    plt.show()

    print(f"text script audio: {sample_data['transcription']}")
    print(f'intent class: {minds["train"].features["intent_class"].names[sample_data["intent_class"]]}')

In [ ]:
### Sample Data 1

sample_data = minds["train"][10]
eda_plots(sample_data)
ipd.Audio(sample_data["audio"]['path'])

In [ ]:
id2label[str(2)]

'app_error'

In [ ]:
minds = minds.train_test_split(test_size=0.2)

In [ ]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [ ]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"]) # apus kolom tidak perlu

In [ ]:
minds["train"][0]

{'audio': {'path': '602bac7505f96973d67944ce.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00024414,
          0.        ,  0.00024414]),
  'sampling_rate': 8000},
 'intent_class': 12}

In [ ]:
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i) # membuat dict untuk maping label ke id
    id2label[str(i)] = label # membuat dict untuk maping id ke label

# Preprocess

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000)) #resample ke 16000 khz
minds["train"][0]

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 'classifier.bias', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# training_args = TrainingArguments(
#   output_dir='latihan_speech_classification',
#   group_by_length=True,
#   per_device_train_batch_size=32,
#   evaluation_strategy="steps",
#   num_train_epochs=30,
#   fp16=True,
#   gradient_checkpointing=True,
#   save_steps=500,
#   eval_steps=500,
#   logging_steps=500,
#   learning_rate=1e-4,
#   weight_decay=0.005,
#   warmup_steps=1000,
#   save_total_limit=2,
#   push_to_hub=True,
# )

In [ ]:
#     # Inisialisasi Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_minds["train"],  # Pastikan Anda telah menyiapkan data terlebih dahulu
#     eval_dataset=encoded_minds["test"],    # Sesuaikan dengan nama dataset evaluasi Anda
#     tokenizer=feature_extractor,
#     compute_metrics=compute_metrics,        # Sesuaikan dengan fungsi komputasi metrik Anda
#     )

In [ ]:
# # Lakukan pelatihan
# trainer.train()

In [ ]:
import optuna

def objective(trial):
    # Tentukan hyperparameter yang akan di-tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-3)
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 10)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16, 32])
    gradient_accumulation_steps = trial.suggest_categorical('gradient_accumulation_steps', [1, 2, 4])

    # Inisialisasi model dengan hyperparameter yang di-tune
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/wav2vec2-base",
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label,
    )

    # Definisi argumen pelatihan
    training_args = TrainingArguments(
        output_dir="latihan_speech_classification",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        per_device_eval_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        push_to_hub=True
        )

    # Inisialisasi Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_minds["train"],  # Pastikan Anda telah menyiapkan data terlebih dahulu
        eval_dataset=encoded_minds["test"],    # Sesuaikan dengan nama dataset evaluasi Anda
        tokenizer=feature_extractor,
        compute_metrics=compute_metrics,        # Sesuaikan dengan fungsi komputasi metrik Anda
    )

    # Lakukan pelatihan
    trainer.train()

    # Hitung dan kembalikan metrik evaluasi (bisa diganti dengan metrik lainnya)
    result = trainer.evaluate()
    accuracy = result['eval_accuracy']
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Cetak hyperparameter terbaik
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2023-12-20 23:14:30,903] A new study created in memory with name: no-name-cea01242-c252-4279-84a6-62c32d927ac4
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 'classifier.bias', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,2.710600,2.882709,0.061947
2,2.678800,2.705130,0.053097
3,2.625500,2.689280,0.026549
4,2.627600,2.698147,0.026549
5,2.635500,2.690507,0.026549
6,2.613500,2.679860,0.026549


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

[I 2023-12-20 23:16:21,690] Trial 0 finished with value: 0.061946902654867256 and parameters: {'learning_rate': 0.003761862441093719, 'num_train_epochs': 6, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 1}. Best is trial 0 with value: 0.061946902654867256.
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 

Epoch,Training Loss,Validation Loss,Accuracy
1,2.691900,2.667952,0.044248
2,2.660300,2.683455,0.044248
3,2.640000,2.707152,0.026549
4,2.634500,2.691821,0.026549
5,2.674000,2.691700,0.026549
6,2.642300,2.697266,0.026549
7,2.634300,2.696954,0.026549


...

[I 2023-12-20 23:37:58,231] Trial 8 finished with value: 0.061946902654867256 and parameters: {'learning_rate': 0.0036036076223927597, 'num_train_epochs': 10, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 4}. Best is trial 7 with value: 0.09734513274336283.
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight', 

Epoch,Training Loss,Validation Loss,Accuracy
1,2.733400,2.799735,0.044248
2,2.682300,2.791387,0.044248
3,2.683000,2.722933,0.044248
4,2.647100,2.701276,0.026549
5,2.621000,2.712545,0.053097


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

[I 2023-12-20 23:39:55,324] Trial 9 finished with value: 0.05309734513274336 and parameters: {'learning_rate': 0.004638336337201983, 'num_train_epochs': 5, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 1}. Best is trial 7 with value: 0.09734513274336283.


Best Hyperparameters: {'learning_rate': 0.003148256598776141, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 4}


## buat model dengan best param

In [ ]:
best_learning_rate = 0.0034193556521543325
best_num_train_epochs = best_params['num_train_epochs']
best_per_device_train_batch_size = best_params['per_device_train_batch_size']
best_gradient_accumulation_steps = best_params['gradient_accumulation_steps']

training_args = TrainingArguments(
    output_dir="latihan_speech_classification",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_per_device_train_batch_size,
    gradient_accumulation_steps=best_gradient_accumulation_steps,
    per_device_eval_batch_size=best_per_device_train_batch_size,
    num_train_epochs=best_num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.888672,0.097345
1,No log,2.656319,0.061947
2,No log,2.729898,0.053097


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=9, training_loss=2.48520999484592, metrics={'train_runtime': 61.203, 'train_samples_per_second': 22.058, 'train_steps_per_second': 0.147, 'total_flos': 9914190144768000.0, 'train_loss': 2.48520999484592, 'epoch': 2.4})